In [ ]:
import pandas as pd
import numpy as np
from copy import copy
import os
import re
import brightway2 as bw

# Read data
Excel sheet can be obtained from https://doi.org/10.1021/acs.est.8b01452

In [ ]:
path = 'data/es8b01452_si_002.xlsx'
sheet_name = 'Overview & LCA-Modeling'
df_raw = pd.read_excel(path, sheet_name = sheet_name, header=2)

# Add missing On columns

In [ ]:
def complete_columns(df):
    
    column_names = list(df.columns)
    indices = [i for i,el in enumerate(column_names)  if 'Activity' in el]
    column_names_complete = copy(column_names)

    n_el_added = 0
    for ind in indices:
        if 'On' not in column_names[ind-1]:
            act_name = column_names[ind]
            act_number = act_name[act_name.find(' ')+1:]
            column_names_complete.insert(ind+n_el_added, 'On ' + act_number)
            n_el_added += 1
        
    df.columns = column_names_complete[:len(column_names)]
    
    return df

In [ ]:
df = complete_columns(df_raw)

# Convert data to brightway database format

In [ ]:
#Constants
# Number of relevant columns in the raw file (df_raw) to extract info about activity
N_ACT_RELEVANT = 11
# Index of the column where activities start
FIRST_ACT_IND = 7
# Number of columns that contain info about one activity
N_COLUMNS_INPUT_ACTIVITY = 5

# Column names for exchanges needed by brightway
EXC_COLUMNS_DICT = {
        'name': 'A', 
        'reference product': 'B', 
        'location': 'C', 
        'amount': 'D', 
        'unit': 'E', 
        'database': 'F', 
        'type': 'G', 
        'categories': 'H',
        'comment': 'I',
    }

# Conversion from type in databases to type that should be in excel file to import a new database
ACTIVITY_TYPE_DICT = {
    'process': 'technosphere',
    'emission': 'biosphere',
}



In [ ]:
def create_df_bw(db_name, n_cutoff_cols = len(EXC_COLUMNS_DICT)+3):
    '''
    Create dataframe for a new database in the Brightway format and add the necessary meta information
    '''
    df = pd.DataFrame([['cutoff', n_cutoff_cols], ['Database', db_name]], columns=list('AB'))
    df = df.append(pd.Series(), ignore_index=True)
    return df

In [ ]:
def compute_act_unit(df):
    if 'Quantity code' in df.keys():
        return df['DB Act 1'].split('(')[1].split(',')[0]
    else:
        return 'CHF'

In [ ]:
def append_activity(df, df_ind):
    '''
    Append activity to the dataframe df in the brightway format
    '''
    # Append empty row
    df = df.append(pd.Series(), ignore_index=True)
    
    # Extract activity information
    act_name = df_ind['Translated name']
    act_unit = compute_act_unit(df)
    
    len_df = len(df)
    
    act_data = [ ['Activity name', act_name],
                 ['reference product',  act_name],
                 ['location', 'CH'],
                 ['amount', 1],
                 ['unit', act_unit] ]
    
    df_act = pd.DataFrame( act_data, 
                           columns=list('AB'),
                           index = np.arange(len_df,len_df+len(act_data)) )
                          
    df = df.append(df_act, sort=False)
    # Append empty row
    df = df.append(pd.Series(), ignore_index=True)
    
    return df

In [ ]:
def add_exchanges_column_names(df):
    col_names = list(EXC_COLUMNS_DICT.keys())
    col_excel_literal = [EXC_COLUMNS_DICT[k] for k in col_names]

    df = df.append(pd.DataFrame(['Exchanges'], columns=['A'], index=[len(df)]), sort=False)
    df = df.append(pd.DataFrame([col_names], columns=col_excel_literal, index=[len(df)]), sort=False)
    return df

In [ ]:
def is_pattern_correct(df_ind_j):
    list_ = list(df_ind_j.index)
    pattern = ['On', 'Activity', 'DB Act', 'CFL Act', 'Amount Act']
    check = [pattern[n] in list_[n] for n in range(N_COLUMNS_INPUT_ACTIVITY)]
    if np.all(check): 
        return 1
    else: 
        return 0

In [ ]:
def add_exchanges(df, df_ind):
    # Add exchanges column names
    df = add_exchanges_column_names(df)
    
    # TODO Add first exchange: here or before?
    
    # Add all exchanges
    n_exchanges = (len(df_ind)-FIRST_ACT_IND) // N_COLUMNS_INPUT_ACTIVITY
    if n_exchanges != (len(df_ind) - FIRST_ACT_IND) / N_COLUMNS_INPUT_ACTIVITY:
        print('smth is not right with exchanges of Activity -> ' + str(df_ind['Translated name']))
    
    ConversionDem2FU = df_ind['ConversionDem2FU']
    skip = 0
    for j in range(1, n_exchanges+1):
        
        start = FIRST_ACT_IND + N_COLUMNS_INPUT_ACTIVITY*(j-1) + skip
        end = start + N_COLUMNS_INPUT_ACTIVITY
        df_ind_j = df_ind[start:end]
        
        #Check that df_ind_j contains <On 1, Activity 1, DB Act 1, CFL Act 1, Amount Act 1> pattern
        flag = 1
        while flag:
            flag_pattern = is_pattern_correct(df_ind_j) 
            if flag_pattern == 1: # we don't need to skip if patter is correct
                flag = 0
            else:
                skip += 1
                start = FIRST_ACT_IND + N_COLUMNS_INPUT_ACTIVITY*(j-1) + skip
                end = start + N_COLUMNS_INPUT_ACTIVITY
                df_ind_j = df_ind[start:end]
        
        df = append_exchange(df, df_ind_j, ConversionDem2FU)
        
    return df

In [ ]:
def create_input_act_dict(act_bw, input_act_amount):
    '''
    Create a dictionary with all info about input activities.
    '''
    input_act_values_dict = {
        'name': act_bw['name'], 
        'reference product': act_bw['reference product'], 
        'location': act_bw['location'], 
        'amount': input_act_amount, 
        'unit': act_bw['unit'], 
        'database': act_bw['database'], 
        # We do not expect type biosphere, but assign it via ACTIVITY_TYPE_DICT anyway 
        # to be sure that we don't encounter them.
        'type': ACTIVITY_TYPE_DICT[act_bw['type']],
    }
    return input_act_values_dict

In [ ]:
def append_exchange(df, df_ind_j, ConversionDem2FU):
    '''
    Note that we use terms `exchange` and `input activity` interchangeably.
    Extract information about the input activity, eg name, unit, location, etc
    '''    
    # Extract the activity number
    k = int(''.join(c for c in df_ind_j.index[0] if c.isdigit()))
    
    input_act_str = df_ind['DB Act ' + str(k)]
    
    # Extract the activity name
    apostrophes = [(m.start(0), m.end(0)) for m in re.finditer("'", input_act_str)]
    if len(apostrophes) == 1:
        ap_start = 0
        ap_end = apostrophes[0][0]
    else:
        ap_start = apostrophes[0][1]
        ap_end = apostrophes[1][0]
    input_act_name = input_act_str[ ap_start:ap_end ]
    
    input_act_unit_loc = input_act_str[ input_act_str.find("(") : input_act_str.find(")")+1 ]
#     input_act_unit_loc_split = [ re.sub('\W+', ' ' , el) for el in input_act_unit_loc.split(',')]
    input_act_unit_loc_split = [ re.sub('[^-A-Za-z0-9]', ' ' , el).rstrip().lstrip() \
                                 for el in input_act_unit_loc.split(',')]
    input_act_unit = input_act_unit_loc_split[0]
    input_act_location = input_act_unit_loc_split[1]
    
    # Extract input activity amount
    input_act_amount = df_ind['CFL Act ' + str(k)] * df_ind['Amount Act ' + str(k)]
    
    # Find this input activity in brightway databases
    input_act_db_code = df_ind['Activity ' + str(k)]
    db_name = input_act_db_code.split("'")[1]
    db = bw.Database(db_name)
    acts_bw = [act for act in db if  input_act_name == act['name'] \
                                 and input_act_unit == act['unit'] \
                                 and input_act_location == act['location']]
    
    try:
        act_bw = acts_bw[0]        
        input_act_values_dict = create_input_act_dict(act_bw, input_act_amount)
        
        # Add comment when activity is not unique
        if len(acts_bw) > 1:
            input_act_values_dict['comment'] = 'TODO: not unique!'
            
    except:
        # Add comment when activity cannot be found
        input_act_values_dict = {}
        input_act_values_dict['name'] = input_act_name
        input_act_values_dict['unit'] = input_act_unit
        input_act_values_dict['location'] = input_act_location
        input_act_values_dict['database'] = db_name
        input_act_values_dict['comment'] = 'TODO: cannot find this activity'
        
    # Add exchange to the dataframe with database in brightway format
    col_names = list(input_act_values_dict.keys())
    col_data  = [input_act_values_dict[m] for m in input_act_values_dict]
    col_excel_literal = [EXC_COLUMNS_DICT[m] for m in col_names]
    
    df = df.append(pd.DataFrame([col_data], columns=col_excel_literal, index=[len(df)]), sort=False)
    
    return df

# Main code

In [ ]:
project = 'GSA for ecoinvent'
bw.projects.set_current(project)

In [ ]:
# Start brightway project that already contains databases
project = 'GSA for ecoinvent'
bw.projects.set_current(project)

consumption_db_name = 'CH consumption 1.0'
df_bw = create_df_bw(consumption_db_name)

act_indices = df_raw.index[df_raw['ConversionDem2FU'].notna()].tolist() # indices of all activities

for ind in act_indices[4:5]:
    # For each row
    df_ind = df_raw.iloc[ind]
    df_ind = df_ind[df_ind.notna()]
    # Add activity
    df_bw = append_activity(df_bw, df_ind[:N_ACT_RELEVANT]) # only pass columns that are relevant to this function 
    # Add exchanges
    df_bw = add_exchanges(df_bw, df_ind)

In [ ]:
write_dir_name = 'write_files'
if not os.path.exists(write_dir_name):
    os.mkdir(write_dir_name)
db_bw_path = write_dir_name + '/' + 'consumption_db.xlsx'
df_bw.to_excel(db_bw_path, index=False, header=False)

# Playground